In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

3.7.9


In [3]:
import chevron
import sys
import os
import copy
import logging
import tensorflow as tf
import torch
import matplotlib.pyplot as plt

from distutils.dir_util import copy_tree

import pandas as pd
import networkx as nx

import time

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
init_modules = copy.deepcopy(list(sys.modules.keys()))
tf.logging.set_verbosity(tf.logging.ERROR)

In [5]:
class redirect_output(object):
    """context manager for reditrecting stdout/err to files"""


    def __init__(self, out=''):
        self.log = open(out, 'w')
        
        self.old_stdout = sys.stdout
        self.old_stderr = sys.stderr
                
    def __enter__(self):
        sys.stdout = self.log
        sys.stderr = self.log
        
    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.old_stdout
        sys.stderr = self.old_stderr
        self.log.close()

In [6]:
def delete_loaded_modules(init_modules):
    to_del = []
    for m in sys.modules.keys(  ):
        if m not in init_modules:
            to_del.append(m)
        
    for m in to_del:
        del(sys.modules[m])

In [7]:
def time_to_str(elapsed):
    
    hours, rem = divmod(elapsed, 3600)
    minutes, seconds = divmod(rem, 60)
    str_ = "{:0>2} seconds".format(int(seconds))
    if minutes > 0 or hours > 0:
        str_ = "{:0>1} minutes and ".format(int(minutes)) + str_
    if hours > 0:
        str_ = "{:0>1} hours ".format(int(hours)) + str_
        
    return str_

In [8]:
def is_a_DATGAN(name):
    if 'TGAN' in name or 'CTGAN' in name:
        return False
    else:
        return True
    
def check_already_trained(dataset, name):
    
    return os.path.isfile('../output/{}/{}/trained.tar.gz'.format(dataset, name)) or os.path.isfile('../output/{}/{}/trained.pickle'.format(dataset, name))

In [9]:
def dag(dataset):

    # personalised graph
    graph = nx.DiGraph()

    if dataset is 'Chicago':
        graph.add_edges_from([
            ("age", "license"),
            ("age", "education_level"),
            ("gender", "work_status"),
            ("education_level", "work_status"),
            ("education_level", "hh_income"),
            ("work_status", "hh_income"),
            ("hh_income", "hh_descr"),
            ("hh_income", "hh_size"),
            ("hh_size", "hh_vehicles"),
            ("hh_size", "hh_bikes"),
            ("work_status", "trip_purpose"),
            ("trip_purpose", "departure_time"),
            ("trip_purpose", "distance"),
            ("travel_dow", "choice"),
            ("distance", "choice"),
            ("departure_time", "choice"),
            ("hh_vehicles", "choice"),
            ("hh_bikes", "choice"),
            ("license", "choice"),
            # Links removed when doing trans red
            ("education_level", "hh_size"),
            ("work_status", "hh_descr"),
            ("work_status", "hh_size"),
            ("hh_income", "hh_bikes"),
            ("hh_income", "hh_vehicles"),
            ("trip_purpose", "choice")
        ])
    elif dataset is 'LPMC':
        graph.add_edges_from([
            ("travel_year", "travel_month"),
            ("travel_date", "day_of_week"),
            ("travel_month", "travel_date"),
            ("travel_month", "driving_traffic_percent"),
            ("travel_month", "day_of_week"),
            ("travel_month", "travel_mode"),
            ("travel_date", "day_of_week"),
            ("day_of_week", "driving_traffic_percent"),
            ("day_of_week", "cost_driving_con_charge"),
            ("day_of_week", "purpose"),
            ("day_of_week", "start_time_linear"),
            ("day_of_week", "travel_mode"),
            ("purpose", "distance"),
            ("purpose", "start_time_linear"),
            ("purpose", "travel_mode"),
            ("start_time_linear", "driving_traffic_percent"),
            ("start_time_linear", "cost_driving_con_charge"),
            ("start_time_linear", "travel_mode"),
            ("car_ownership", "fueltype"),
            ("car_ownership", "driving_license"),
            ("car_ownership", "travel_mode"),
            ("fueltype", "cost_driving_con_charge"),
            ("fueltype", "cost_driving_fuel"),
            ("female", "driving_license"),
            ("female", "travel_mode"),
            ("age", "bus_scale"),
            ("age", "driving_license"),
            ("age", "faretype"),
            ("age", "travel_mode"),
            ("driving_license", "travel_mode"),
            ("faretype", "cost_transit"),
            ("faretype", "bus_scale"),
            ("faretype", "travel_mode"),
            ("bus_scale", "cost_transit"),
            ("distance", "cost_driving_fuel"),
            ("distance", "dur_driving"),
            ("distance", "dur_walking"),
            ("distance", "dur_cycling"),
            ("distance", "dur_pt_access"),
            ("distance", "dur_pt_rail"),
            ("distance", "dur_pt_bus"),
            ("distance", "dur_pt_int"),
            ("distance", "pt_n_interchanges"),
            ("distance", "travel_mode"),
            ("pt_n_interchanges", "dur_pt_rail"),
            ("pt_n_interchanges", "dur_pt_bus"),
            ("pt_n_interchanges", "dur_pt_int"),
            ("pt_n_interchanges", "cost_transit"),
            ("driving_traffic_percent", "cost_driving_con_charge"),
            ("driving_traffic_percent", "travel_mode"),
            ("cost_driving_fuel", "cost_driving_con_charge"),
            ("cost_driving_fuel", "travel_mode"),
            ("cost_driving_con_charge", "travel_mode"),
            ("dur_driving", "travel_mode"),
            ("dur_walking", "travel_mode"),
            ("dur_cycling", "travel_mode"),
            ("dur_pt_access", "travel_mode"),
            ("dur_pt_rail", "cost_transit"),
            ("dur_pt_rail", "travel_mode"),
            ("dur_pt_bus", "cost_transit"),
            ("dur_pt_bus", "travel_mode"),
            ("dur_pt_int", "travel_mode"),
            ("cost_transit", "travel_mode")
        ])
        graph.add_node("travel_month")
        
    return graph

In [10]:
def train_DATGAN(dataset, loss, name, nt='WI'):
    
    df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

    if dataset is 'Chicago':
        continuous_columns = ["distance", "age", "departure_time"]
    elif dataset is 'LPMC':
        continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
                              'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit',
                              'cost_driving_fuel', 'driving_traffic_percent']

    if loss == 'WGAN':
        from modules.datgan import DATWGAN as LIB
    elif loss == 'SGAN':
        from modules.datgan import DATSGAN as LIB
    elif loss == 'WGGP':
        from modules.datgan import DATWGANGP as LIB

    output_folder = '../output/{}/{}/'.format(dataset, name)

    datgan = LIB(continuous_columns, max_epoch=1000, batch_size=500, 
                 output=output_folder, gpu=0, noisy_training=nt)

    datgan.fit(df, dag(dataset))

    datgan.save('trained', force=True)
    
def train_TGAN(dataset, name):
    
    df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

    if dataset is 'Chicago':
        continuous_columns = [3, 10, 14]
    elif dataset is 'LPMC':
        continuous_columns = [9, 10, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 26]
        
    from tgan.model import TGANModel

    output_folder = '../output/{}/{}/'.format(dataset, name)
    
    bs = 500
    steps_per_epoch = max(len(df) // bs, 1)
    
    tgan = TGANModel(continuous_columns, max_epoch=1000, steps_per_epoch=steps_per_epoch, 
                     batch_size=bs, output=output_folder, gpu=0)

    tgan.fit(df)

    tgan.save(output_folder + 'trained.pickle', force=True)
    
def train_CTGAN(dataset, name):
    
    df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)
    
    if dataset is 'Chicago':
        discrete_columns = [
            'choice',
            'travel_dow',
            'trip_purpose',
            'hh_vehicles',
            'hh_size',
            'hh_bikes',
            'hh_descr',
            'hh_income',
            'gender',
            'license',
            'education_level',
            'work_status'
        ]
    elif dataset is 'LPMC':
        discrete_columns = [
            'travel_mode',
             'purpose',
             'fueltype',
             'faretype',
             'bus_scale',
             'travel_year',
             'travel_month',
             'travel_date',
             'day_of_week',
             'female',
             'driving_license',
             'car_ownership',
             'pt_n_interchanges',
             'cost_driving_con_charge'
        ]
    from ctgan import CTGANSynthesizer
        
    output_folder = '../output/{}/{}/'.format(dataset, name)
    os.makedirs(output_folder)
    
    ctgan = CTGANSynthesizer(verbose=True, cuda=True, batch_size=500)
    
    ctgan.fit(df, discrete_columns, epochs=1000)
    
    ctgan.save(output_folder + 'trained.pickle')

In [11]:
dataset = 'Chicago'
n_models = 5

models = []#['TGAN', 'CTGAN']

reuse_data = False

for i in ['WGAN', 'WGGP', 'SGAN']:
    for j in ['WI', 'OR', 'WO']:
        models.append('{}_{}'.format(i,j))
        
if n_models > 1:
    tmp = []
    
    for i in range(n_models):
        for m in models:
            tmp.append(m + '_{:0>2d}'.format(i+1))
        
    tmp.sort()
    models = tmp

In [12]:
for i, m in enumerate(models):
    
    if check_already_trained(dataset, m):
        print("Model \033[1m{}\033[0m ({}/{}) has already been trained.".format(m, i+1, len(models)))
        
    else:
    
        print("\rTraining model \033[1m{}\033[0m ({}/{}) ... ".format(m, i+1, len(models)), end="")
        
        # Cannot delete tensorflow modules sadly =(
        tf.reset_default_graph()

        delete_loaded_modules(init_modules)

        if reuse_data and is_a_DATGAN(m):
            copy_tree('../output/{}/{}/data'.format(dataset, 'WGAN_WI'), '../output/{}/{}/data'.format(dataset, m))

        start_time = time.time()

        if 'CTGAN' in m:
            with redirect_output('training.log'):
                train_CTGAN(dataset, m)
        elif 'TGAN' in m:
            with redirect_output('training.log'):
                train_TGAN(dataset, m)
        elif is_a_DATGAN(m):
            with redirect_output('training.log'):
                train_DATGAN(dataset, m.split('_')[0], m, m.split('_')[1])

        elapsed = time.time() - start_time

        time_taken = time_to_str(elapsed)

        print("Done in {}.".format(time_taken))

        for handler in logging.getLogger('tensorpack').handlers:
            handler.close()

        logging.getLogger('tensorpack').handlers = []

print("FINISHED!")

Model SGAN_OR_01 (1/45) has already been trained.
Model SGAN_OR_02 (2/45) has already been trained.
Model SGAN_OR_03 (3/45) has already been trained.
Model SGAN_OR_04 (4/45) has already been trained.
Model SGAN_OR_05 (5/45) has already been trained.
Model SGAN_WI_01 (6/45) has already been trained.
Model SGAN_WI_02 (7/45) has already been trained.
Model SGAN_WI_03 (8/45) has already been trained.
Model SGAN_WI_04 (9/45) has already been trained.
Model SGAN_WI_05 (10/45) has already been trained.
Model SGAN_WO_01 (11/45) has already been trained.
Model SGAN_WO_02 (12/45) has already been trained.
Model SGAN_WO_03 (13/45) has already been trained.
Model SGAN_WO_04 (14/45) has already been trained.
Model SGAN_WO_05 (15/45) has already been trained.
Model WGAN_OR_01 (16/45) has already been trained.
Model WGAN_OR_02 (17/45) has already been trained.
Model WGAN_OR_03 (18/45) has already been trained.
Model WGAN_OR_04 (19/45) has already been trained.
Model WGAN_OR_05 (20/45) has already bee